In [ ]:
import torch
import torch.nn.functional as F
from torchvision.models import resnet50  # Example model

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.model.eval()
        self.register_hooks()

    def save_gradients(self, grad):
        self.gradients = grad

    def register_hooks(self):
        def forward_hook(module, input, output):
            self.features = output

        def backward_hook(module, grad_in, grad_out):
            self.save_gradients(grad_out[0])

        for name, module in self.model.named_modules():
            if name == self.target_layer:
                module.register_forward_hook(forward_hook)
                module.register_backward_hook(backward_hook)

    def __call__(self, x, index=None):
        output = self.model(x)
        if index is None:
            index = output.argmax(dim=1)
        
        one_hot = torch.zeros_like(output)
        one_hot[range(one_hot.shape[0]), index] = 1
        
        self.model.zero_grad()
        output.backward(gradient=one_hot, retain_graph=True)
        
        with torch.no_grad():
            weights = self.gradients.mean(dim=[2, 3], keepdim=True)
            grad_cam = F.relu((weights * self.features).sum(dim=1)).squeeze(0)
        
        return grad_cam

# Example usage
model = resnet50(pretrained=True)
grad_cam = GradCAM(model, target_layer='layer4')  # Specify the target layer
# Prepare your input image
# input_image = ...
# grad_cam_map = grad_cam(input_image)


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet
from torchvision.models import inception_v3, xception  # Placeholder for actual EfficientNet models

# Assuming EfficientNet B1, B2, B3 are similar to these placeholder models for demonstration
models = {
    'B1': EfficientNet.from_pretrained('efficientnet-b1'),
    'B2': EfficientNet.from_pretrained('efficientnet-b2'),
    'B3': EfficientNet.from_pretrained('efficientnet-b3'),
}

# Modify models for feature extraction
for model in models.values():
    model._fc = nn.Identity()

# Data augmentation and normalization for training
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = ImageFolder(root='/path/to/train/data', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Ensemble Model
class EnsembleEfficientNet(nn.Module):
    def __init__(self, models):
        super(EnsembleEfficientNet, self).__init__()
        self.models = nn.ModuleList(models)
        self.classifier = nn.Linear(sum([model._fc.in_features for model in models]), 2)  # Assuming binary classification

    def forward(self, x):
        features = torch.cat([model(x) for model in self.models], dim=1)
        out = self.classifier(features)
        return out

# Initialize ensemble model
ensemble_model = EnsembleEfficientNet(models=[models['B1'], models['B2'], models['B3']])

# Training specifics
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ensemble_model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):  # Number of epochs
    ensemble_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')

# Note: Grad-CAM implementation and specific fine-tuning strategies as described in the document would require additional code.
import torch
import torch.nn.functional as F
from torchvision.models import resnet50  # Example model

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.model.eval()
        self.register_hooks()

    def save_gradients(self, grad):
        self.gradients = grad

    def register_hooks(self):
        def forward_hook(module, input, output):
            self.features = output

        def backward_hook(module, grad_in, grad_out):
            self.save_gradients(grad_out[0])

        for name, module in self.model.named_modules():
            if name == self.target_layer:
                module.register_forward_hook(forward_hook)
                module.register_backward_hook(backward_hook)

    def __call__(self, x, index=None):
        output = self.model(x)
        if index is None:
            index = output.argmax(dim=1)
        
        one_hot = torch.zeros_like(output)
        one_hot[range(one_hot.shape[0]), index] = 1
        
        self.model.zero_grad()
        output.backward(gradient=one_hot, retain_graph=True)
        
        with torch.no_grad():
            weights = self.gradients.mean(dim=[2, 3], keepdim=True)
            grad_cam = F.relu((weights * self.features).sum(dim=1)).squeeze(0)
        
        return grad_cam

# Example usage
model = resnet50(pretrained=True)
grad_cam = GradCAM(model, target_layer='layer4')  # Specify the target layer
# Prepare your input image
# input_image = ...
# grad_cam_map = grad_cam(input_image)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet

class EnsembleGradCAM:
    def __init__(self, models, target_layers):
        self.models = models
        self.target_layers = target_layers
        self.features = []
        self.gradients = []
        self.register_hooks()

    def register_hooks(self):
        for model, target_layer in zip(self.models, self.target_layers):
            model.eval()

            def forward_hook(module, input, output):
                self.features.append(output)

            def backward_hook(module, grad_in, grad_out):
                self.gradients.append(grad_out[0])

            layer = dict([*model.named_modules()])[target_layer]
            layer.register_forward_hook(forward_hook)
            layer.register_backward_hook(backward_hook)

    def __call__(self, x):
        # Forward pass
        for model in self.models:
            _ = model(x)
        
        # Combine feature maps
        combined_features = torch.mean(torch.stack(self.features), dim=0)
        # Generate a pseudo output for backprop
        score = combined_features.mean(dim=[2, 3], keepdim=True).sum()

        # Backward pass
        self.models[0].zero_grad()  # Assuming all models share parameters
        score.backward(retain_graph=True)

        # Combine gradients
        combined_gradients = torch.mean(torch.stack(self.gradients), dim=0)
        
        # Apply ReLU to the combined feature maps
        weights = F.relu(combined_gradients)
        grad_cam = torch.mul(combined_features, weights).sum(dim=1).clamp(min=0)

        return grad_cam

# Initialize models
models = [EfficientNet.from_pretrained(f'efficientnet-b{i}') for i in range(1, 4)]
target_layers = ['_blocks.22', '_blocks.22', '_blocks.22']  # Example target layers

# Initialize EnsembleGradCAM
ensemble_grad_cam = EnsembleGradCAM(models, target_layers)

# Prepare your input image and apply EnsembleGradCAM
# input_image = ...
# grad_cam_map = ensemble_grad_cam(input_image)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet


data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root='/path/to/train/data', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


class EnsembleEfficientNet(nn.Module):
    def __init__(self, models):
        super(EnsembleEfficientNet, self).__init__()
        self.models = nn.ModuleList(models)
        # Assuming binary classification for simplicity
        self.classifier = nn.Linear(sum([model._fc.in_features for model in models]), 2)

    def forward(self, x):
        features = torch.cat([model(x) for model in self.models], dim=1)
        out = self.classifier(features)
        return out

models = {
    'B1': EfficientNet.from_pretrained('efficientnet-b1'),
    'B2': EfficientNet.from_pretrained('efficientnet-b2'),
    'B3': EfficientNet.from_pretrained('efficientnet-b3'),
}

# Modify models for feature extraction
for model in models.values():
    model._fc = nn.Identity()

ensemble_model = EnsembleEfficientNet(models=[models['B1'], models['B2'], models['B3']])


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ensemble_model.parameters(), lr=0.001)

for epoch in range(10):  # Adjust epochs as needed
    ensemble_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')

    
class EnsembleGradCAM:
    def __init__(self, models, target_layers):
        """
        Initializes the EnsembleGradCAM.
        
        :param models: A list of models to be included in the ensemble.
        :param target_layers: A list of target layer names for each model where gradients should be captured.
        """
        self.models = models
        self.target_layers = target_layers
        self.gradients = []
        self.features = []

        self.register_hooks()

    def register_hooks(self):
        """
        Registers forward and backward hooks on the target layers of each model.
        """
        for model, target_layer in zip(self.models, self.target_layers):
            model.eval()  # Set model to evaluation mode to disable dropout, etc.
            
            def forward_hook(module, input, output):
                self.features.append(output)
                
            def backward_hook(module, input, output):
                self.gradients.append(output[0])
                
            # Retrieve the target layer
            layer = dict([*model.named_modules()])[target_layer]
            # Register hooks
            layer.register_forward_hook(forward_hook)
            layer.register_backward_hook(backward_hook)

    def __call__(self, x):
        """
        Performs a forward pass and computes the GradCAM for the ensemble.
        
        :param x: Input tensor.
        :return: A tensor representing the GradCAM output.
        """
        self.features = []
        self.gradients = []
        
        # Forward pass through each model
        for model in self.models:
            _ = model(x)
        
        # Aggregate and process the features and gradients here as necessary
        # This can involve averaging or another method of combining the information
        # from different models
        
        # Example: Averaging the features and gradients
        # Note: This is a simplification. You might need a more sophisticated method to combine these.
        avg_features = torch.mean(torch.stack(self.features), dim=0)
        avg_gradients = torch.mean(torch.stack(self.gradients), dim=0)

        # Use the processed features and gradients to compute GradCAM
        # This is a simplified version; adapt as necessary for your use case.
        gcam = self.compute_gradcam(avg_features, avg_gradients)
        
        return gcam

    def compute_gradcam(self, features, gradients):
        """
        Computes the GradCAM based on features and gradients.
        
        :param features: Aggregated features from the forward hook.
        :param gradients: Aggregated gradients from the backward hook.
        :return: GradCAM output.
        """
        # Weight the channels by corresponding gradients
        weights = torch.mean(gradients, dim=[2, 3], keepdim=True)
        gcam = torch.mul(features, weights).sum(dim=1, keepdim=True)
        gcam = F.relu(gcam)  # Apply ReLU to the GradCAM
        
        # Normalize the GradCAM
        gcam = gcam - gcam.min()
        gcam = gcam / gcam.max()
        
        return gcam
    
    
import torch
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet

class EnsembleGradCAM:
    def __init__(self, models, target_layers):
        self.models = models
        self.target_layers = target_layers
        self.features = []
        self.gradients = []
        self.register_hooks()

    def register_hooks(self):
        for model, target_layer in zip(self.models, self.target_layers):
            model.eval()

            def forward_hook(module, input, output):
                self.features.append(output)

            def backward_hook(module, grad_in, grad_out):
                self.gradients.append(grad_out[0])

            layer = dict([*model.named_modules()])[target_layer]
            layer.register_forward_hook(forward_hook)
            layer.register_backward_hook(backward_hook)

    def __call__(self, x):
        # Forward pass
        for model in self.models:
            _ = model(x)
        
        # Combine feature maps
        combined_features = torch.mean(torch.stack(self.features), dim=0)
        # Generate a pseudo output for backprop
        score = combined_features.mean(dim=[2, 3], keepdim=True).sum()

        # Backward pass
        self.models[0].zero_grad()  # Assuming all models share parameters
        score.backward(retain_graph=True)

        # Combine gradients
        combined_gradients = torch.mean(torch.stack(self.gradients), dim=0)
        
        # Apply ReLU to the combined feature maps
        weights = F.relu(combined_gradients)
        grad_cam = torch.mul(combined_features, weights).sum(dim=1).clamp(min=0)

        return grad_cam

# Initialize models
models = [EfficientNet.from_pretrained(f'efficientnet-b{i}') for i in range(1, 4)]
target_layers = ['_blocks.22', '_blocks.22', '_blocks.22']  # Example target layers

# Initialize EnsembleGradCAM
ensemble_grad_cam = EnsembleGradCAM(models, target_layers)
